In [6]:
from pydantic import BaseModel
from typing import Optional
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import RDF, RDFS
from pyfuseki import FusekiUpdate
from rdflib.plugins.stores import sparqlstore
from rdflib.namespace import XSD
from datetime import datetime
from src.schemas.settings import Settings


In [2]:
class BfElement(BaseModel):
    value: str
    label: str

class AdminMetadata(BaseModel):
    creationDate: str
    status: BfElement
    descriptionConventions: BfElement
    identifiedBy: str


class BfWork(BaseModel):
    resourceType: list[BfElement]
    adminMetadata: AdminMetadata


In [3]:
req = {
    "resourceType": [
        {
            "value": "Work",
            "label": "Obra"
        },
        {
            "value": "Text",
            "label": "Texto"
        }
    ],
    "adminMetadata": {
        "creationDate": "2024-03-09",
        "status": {
            "value": "http://id.loc.gov/vocabulary/mstatus/n",
            "label": "Novo"
        },
        "descriptionConventions": {
            "value": "http://id.loc.gov/vocabulary/descriptionConventions/aacr",
            "label": "AACr"
        },
        "identifiedBy": "1"
    }
}

In [7]:
request = BfWork(**req)

settings = Settings()

In [29]:
request.adminMetadata.status.label

'Novo'

In [13]:
now = datetime.now()
formatted_date = now.strftime("%Y-%m-%dT%H:%M:%S")
formatted_date

'2024-03-09T17:40:10'

In [40]:
def MakeGraphWork(request):

    now = datetime.now()   

    BF = Namespace("http://id.loc.gov/ontologies/bibframe/")

    identifier = f"https://bibliokeia/works/{request.adminMetadata.identifiedBy}"
    resource = URIRef(identifier)

    g = Graph(identifier=identifier)
    g.bind('bf', BF)

    
    # resourceType
    for i in request.resourceType:
        bfType = URIRef(f"http://id.loc.gov/ontologies/bibframe/{i.value}")
        g.add((resource, RDF.type, bfType))

    # AdminMetadata
    adminMetadata = BNode()
    g.add((resource, BF.adminMetadata, adminMetadata))
    g.add((adminMetadata, RDF.type, BF.AdminMetadata ))
    # assigner
    assigner_uri = URIRef(settings.organization_loc_uri)
    g.add((assigner_uri, RDF.type, BF.Organization))
    g.add((assigner_uri, RDFS.label, Literal(settings.organization)))
    g.add((adminMetadata, BF.assigner, assigner_uri ))
    # creationDate
    formatted_date = now.strftime("%Y-%m-%d")
    g.add((adminMetadata, BF.creationDate, Literal(formatted_date, datatype=XSD.date) ))
    # descriptionConventions
    descriptionConventions = URIRef(request.adminMetadata.descriptionConventions.value)
    g.add((descriptionConventions, RDF.type, BF.DescriptionConventions ))
    g.add((descriptionConventions, RDFS.label, Literal(request.adminMetadata.descriptionConventions.label) ))
    g.add((adminMetadata, BF.descriptionConventions, descriptionConventions ))
    # generationProcess
    formatted_dateTime = now.strftime("%Y-%m-%dT%H:%M:%S")
    generationProcess = BNode() 
    g.add((adminMetadata, BF.generationProcess, generationProcess))
    g.add((generationProcess, RDF.type, BF.GenerationProcess))
    g.add((generationProcess, RDFS.label, Literal(settings.app_name) ))
    g.add((generationProcess, BF.generationDate, Literal(formatted_dateTime, datatype=XSD.dateTime) ))
    # identifiedBy
    identifiedBy =  BNode() 
    g.add((adminMetadata, BF.identifiedBy, identifiedBy))
    g.add((identifiedBy, RDF.type, BF.Local))
    g.add((identifiedBy, BF.assigner, assigner_uri))
    g.add((identifiedBy, RDF.value, Literal(request.adminMetadata.identifiedBy)))
    # status
    status = URIRef(request.adminMetadata.status.value)
    g.add((status, RDF.type, BF.Status))
    g.add((status, RDFS.label, Literal(request.adminMetadata.status.label)))
    g.add((adminMetadata, BF.status, status))
 

    graph = g.serialize(format='ttl')
    # g.serialize(destination="work_test.ttl", format='ttl')

    return graph

def MakeSparql(graph):
    graph = graph.split("\n\n")
    graph = "\n\n".join(graph[1:])

    graph = f"""
        PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>  
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

        INSERT DATA {{
            GRAPH <https://bibliokeia/works/10> 
            {{
                {graph} }} }} """

    return graph



graph = MakeGraphWork(request)
sparql = MakeSparql(graph)
print(sparql)


        PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>  
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

        INSERT DATA {
            GRAPH <https://bibliokeia/works/10> 
            {
                <https://bibliokeia/works/1> a bf:Text,
        bf:Work ;
    bf:adminMetadata [ a bf:AdminMetadata ;
            bf:assigner <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
            bf:creationDate "2024-03-09"^^xsd:date ;
            bf:descriptionConventions <http://id.loc.gov/vocabulary/descriptionConventions/aacr> ;
            bf:generationProcess [ a bf:GenerationProcess ;
                    rdfs:label "BiblioKeia" ;
                    bf:generationDate "2024-03-09T18:06:06"^^xsd:dateTime ] ;
            bf:identifiedBy [ a bf:Local ;
                    bf:assigner <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
   

In [39]:
fuseki = FusekiUpdate("http://localhost:3030" , 'bk') 
response = fuseki.run_sparql(sparql) 
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [48]:
update_endpoint = 'http://localhost:3030/bk/update'
query_endpoint = 'http://localhost:3030/bk/query'

store = sparqlstore.SPARQLUpdateStore(query_endpoint=query_endpoint, update_endpoint=update_endpoint)
# store.open((query_endpoint, update_endpoint))

{'statusCode': 200, 'message': 'Update succeeded'}

In [55]:
url = 'http://localhost:3030/bk/data?graph=https:%2F%2Fbibliokeia%2Fworks%2F10'
g = Graph()
g.parse(url)

<Graph identifier=N670023130edb414d90f78f4a3ab110a8 (<class 'rdflib.graph.Graph'>)>

In [56]:
print(g.serialize(format='ttl'))

@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .

<https://bibliokeia/works/10> a bf:Text,
        bf:Work .


